In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from __future__ import print_function

import os

import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import numpy as np
import pandas as pd
# from deepviz.guided_backprop import GuidedBackprop
# from sklearn.cross_validation import train_test_split
from sklearn.metrics.ranking import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from deepviz.guided_backprop import GuidedBackprop

try:
    import keras
    from imblearn.keras import balanced_batch_generator
    from imblearn.under_sampling import NearMiss
    import keras.backend as K
    from keras.models import Sequential
    from keras.models import model_from_json
    from keras.layers import Dense, Dropout, Activation
    from keras.optimizers import RMSprop
    from keras.optimizers import Adam
    from keras.optimizers import Adagrad
    from keras.optimizers import SGD
    from keras.callbacks import LambdaCallback, ReduceLROnPlateau, ModelCheckpoint
    from keras.layers.core import Lambda
    from keras.losses import categorical_crossentropy
    import tensorflow as tf
    from keras import regularizers

except:
    print("Keras not found")
    
np.random.seed(42)


def my_basename(path):
    return os.path.splitext(os.path.split(path)[1])[0]

file_dir=r'C:/Users/Mark.Rademaker/PycharmProjects/InternshipNaturalis/trait-geo-diverse-dl/concept proof'

Using TensorFlow backend.


In [4]:
#access file with list of taxa names
taxa=pd.read_csv(file_dir+"/data/spec_filtered/taxa.txt",header=None)
taxa.columns=["taxon"] 

In [38]:
#locations to predict (land areas)
pred_loc=pd.read_csv(file_dir + "/data/GIS/world_locations_to_predict.csv")
lon = pred_loc["decimal_longitude"]
lat = pred_loc["decimal_latitude"]
lat = pd.Series.tolist(lat)
lon = pd.Series.tolist(lon)



#input stacked tiff
src=rasterio.open(file_dir+'/data/GIS/env_stacked/ENVIREM_BIOCLIM_stacked.tif')
profile=src.profile
band=src.read(1) #as np array


#for species in taxa["taxon"][1:]:
spec="Capreolus_capreolus"
new_band=band.copy() # to write results in

#load json and create model
json_file = open(file_dir+'/results/DNN_model/%s_model.json'%spec,'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

#load weights into new model
loaded_model.load_weights(file_dir+'/results/DNN_model/%s_model.h5'%spec)
print('Loaded model from disk')

#compile model
loaded_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])



for i in range(1, len(pred_loc)):
    X=[]
    row, col = src.index(lon[i], lat[i]) # spatial --> image coordinates  
    for j in range (1, 42):
        band = src.read(j)
        #subset of array excluding no data values (excluding sea)
        a = np.where(band > -1000)
        mean = np.mean(a)
        std_dev = np.std(a)
        value = band[row,col]
        value = ((value - mean) / std_dev) #scale value
        # What is the value?
        X.append(value) #append values
    if all(i >= -1000 for i in X) == True:#exclude cells that still contain no-data values (probably bordering the sea)
        X=[X]
        X=np.vstack(X)
        print(X)
        #new_X=np.asarray(X)
        #print(new_X)
        #print(new_X.shape[1])
        #print(new_X.shape)
        #predict new value
        new_value = loaded_model.predict(x=X,batch_size=1,verbose=0) ###predict output value
        print('predicted value ', new_value)
        #new_band[row,col]=new_value

#write to file
#with rasterio.open(file_dir+'/results/DNN_global_predictions/%s_predicted_map.tif'%spec, 'w', **profile) as dst:
   # dst.write(new_band.astype(rasterio.float32), 1)



Loaded model from disk
[[-1.33731767 -1.21955912 -1.33883108 -1.21439678 -1.28277011 -1.3341442
  -1.30599305 -1.32190686 -1.31185788 -1.33417922 -1.56962562 -1.35078085
  -1.33417922 -1.33417922 -1.32913722  0.20792778 -1.30032956 -1.31088624
  -1.84012961 -1.04092397 -1.28230681 -1.34153406 -1.48824024 -1.34118199
  -1.60553669 -1.20984483 -1.31489031 -1.32889638 -1.29913349 -1.28425205
  -1.31401493 -1.29128723 -1.31229555 -1.2737803  -1.31929832  9.13321661
  -1.30266674 -1.64755334 -0.98929262 -1.36130947 -1.52760965]]
predicted value  [[1.0000000e+00 1.5605019e-11]]


KeyboardInterrupt: 